## Download jar if not already present

In [1]:
import sys
import os

current_dir = "./"

sys.path.insert(0, os.path.join(current_dir, "python"))

In [2]:
from download_iceberg_jar import get_jar_file_path_and_name, download_jar

jar_dir, jar_file_name = get_jar_file_path_and_name()
jar_full_path = os.path.join(jar_dir, jar_file_name)

if os.path.isfile(jar_full_path):
    print("JAR file already exists")
else:
    download_jar(jar_dir, jar_file_name)


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os
from download_iceberg_jar import get_jar_file_path_and_name
from warehouse import ensure_warehouse, get_warehouse_top_level_dir

ensure_warehouse()

jar_dir, jar_file_name = get_jar_file_path_and_name()

conf = SparkConf()
conf = conf.set("spark.deploy.mode", "client")
conf.set(
    "spark.jars",
    os.path.join(jar_dir, jar_file_name)
)
conf.set("spark.executor.memory", "2g")
conf.set("spark.driver.memory", "2g")
conf.set("spark.driver.maxResultSize", "1g")
conf.set(
    "spark.sql.extensions",
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
)
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
conf.set("spark.sql.catalog.spark_catalog.type", "hive")
conf.set("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
conf.set("spark.sql.catalog.iceberg.type", "hive")
conf.set("spark.sql.catalog.iceberg.uri", "thrift://localhost:9083")
conf.set("spark.sql.catalog.iceberg.warehouse", get_warehouse_top_level_dir())
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
session = (
    SparkSession.builder.master("local[1]").appName("iceberg-rs").config(conf=conf).getOrCreate()
)


22/10/15 15:33:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
session.sql("create database if not exists iceberg.db1")

DataFrame[]

In [5]:
# Are interval types supported by Iceberg?
# Array, Struct and Map types will be tested later
sql = """
CREATE TABLE iceberg.db1.db1v1table1 (
    byte0 BYTE COMMENT 'byte data type column, rev 0',
    bool0 BOOLEAN COMMENT 'boolean data type column, rev 0',
    short0 SHORT COMMENT 'short data type column, rev 0',
    int0 INT COMMENT 'integer data type column, rev 0', 
    long0 LONG COMMENT 'long data type column, rev 0', 
    float0 FLOAT COMMENT 'float data type column, rev 0',
    double0 DOUBLE COMMENT 'double data type column, rev 0',
    date0 DATE COMMENT 'date data type column, rev 0',
    timestamp0 TIMESTAMP COMMENT 'timestamp data type column, rev 0',
    string0 STRING COMMENT 'string data type column, rev 0',
    binary0 BINARY COMMENT 'binary data type column, rev 0',
    decimal0 DECIMAL COMMENT 'decimal data type column, rev 0'
) 
USING ICEBERG
PARTITIONED BY (bucket(16, byte0), days(timestamp0), string0)
TBLPROPERTIES ('format-version'='1')
"""

session.sql(sql).toPandas()

""


In [6]:
# Are interval types supported by Iceberg?
# Array, Struct and Map types will be tested later
sql = """
CREATE TABLE iceberg.db1.db1v2table1 (
    byte0 BYTE COMMENT 'byte data type column, rev 0',
    bool0 BOOLEAN COMMENT 'boolean data type column, rev 0',
    short0 SHORT COMMENT 'short data type column, rev 0',
    int0 INT COMMENT 'integer data type column, rev 0', 
    long0 LONG COMMENT 'long data type column, rev 0', 
    float0 FLOAT COMMENT 'float data type column, rev 0',
    double0 DOUBLE COMMENT 'double data type column, rev 0',
    date0 DATE COMMENT 'date data type column, rev 0',
    timestamp0 TIMESTAMP COMMENT 'timestamp data type column, rev 0',
    string0 STRING COMMENT 'string data type column, rev 0',
    binary0 BINARY COMMENT 'binary data type column, rev 0',
    decimal0 DECIMAL COMMENT 'decimal data type column, rev 0'
) 
USING ICEBERG
PARTITIONED BY (bucket(16, byte0), days(timestamp0), string0)
TBLPROPERTIES ('format-version'='2')
"""

session.sql(sql).toPandas()

""


In [7]:
from pyspark.sql import Row
from datetime import datetime, date

data = [
    {
        "byte0": 123, "bool0": True, "short0": 121, "int0": 65532, "long0": 989897989, 
        "float0": 1.23, "double0": 1.39323232, "date0": date.today(), "timestamp0": datetime.utcnow(),
        "string0": "this is a string", "binary0": b"this is a binary", "decimal0": 11.33
    },
    {
        "byte0": 223, "bool0": False, "short0": 121, "int0": 65532, "long0": 989897989, 
        "float0": 1.23, "double0": 1.39323232, "date0": date.today(), "timestamp0": datetime.utcnow(),
        "string0": "this is another string", "binary0": b"this is another binary", "decimal0": 11.33
    },
]

df = session.createDataFrame(Row(**x) for x in data)

In [8]:
df.writeTo("iceberg.db1.db1v1table1").append()
df.writeTo("iceberg.db1.db1v2table1").append()